In [0]:
# Este código se ejecuta en una celda de un Notebook de Databricks
# Su responsabilidad es crear la tabla de la Capa de Oro (Gold),
# aplicando una lógica de unión avanzada para maximizar la categorización.
from pyspark.sql.functions import col, when, upper, regexp_extract, lit

# 1. --- Configuración de Tablas ---
silver_table = "workspace.tecnomundo_data_processed.reporte_de_ventas_por_articulos_2_cleaned"
dimension_table = "workspace.tecnomundo_data_dimensions.category"
gold_table = "workspace.tecnomundo_data_processed.reporte_de_ventas_por_articulos_2_categorized"
unmatched_products_table = "workspace.tecnomundo_data_reporting.productos_sin_categoria"

print(f"Tabla de Origen (Plata): {silver_table}")
print(f"Tabla de Dimensiones: {dimension_table}")
print(f"Tabla de Destino (Oro): {gold_table}")
print(f"Tabla de Reporte (Productos no encontrados): {unmatched_products_table}")


# 2. --- Lectura de las Tablas de Origen ---
print(f"\nLeyendo datos desde la Capa de Plata: '{silver_table}'...")
df_silver = spark.table(silver_table)

print(f"Leyendo datos desde la tabla de Dimensiones: '{dimension_table}'...")
# CORRECCIÓN: Se renombra la columna correcta ('nombre_del_artculo') para evitar ambigüedades.
df_dimensions = (spark.table(dimension_table)
                 .withColumnRenamed("nombre_del_artculo", "dim_nombre_producto")
                 .withColumnRenamed("categoria", "dim_categoria"))


# 3. --- Lógica de Unión en Múltiples Pasos ---
print("\nIniciando proceso de categorización en múltiples pasos...")

# Paso 3a: Estandarizar la clave en la tabla de ventas para el segundo intento
# Creamos una columna temporal con la versión del código sin prefijo.
df_silver_with_keys = df_silver.withColumn("codigo_sin_prefijo",
    upper(
        when(col("codigo_producto").rlike("^[A-Z]\\d{2}-"), regexp_extract(col("codigo_producto"), r'-([^-]*)$', 1))
        .otherwise(col("codigo_producto"))
    )
)

# Intento 1: Unir por coincidencia exacta del código original
print("  - Intento 1: Uniendo por coincidencia exacta del código...")
df_joined_exact = df_silver_with_keys.join(
    df_dimensions,
    df_silver_with_keys.codigo_producto == df_dimensions.codigo_producto,
    "left"
).drop(df_dimensions.codigo_producto) # Se elimina la clave duplicada

# Separamos los que sí se categorizaron de los que no
df_matched_1 = df_joined_exact.filter(col("dim_categoria").isNotNull())
df_unmatched_1 = df_joined_exact.filter(col("dim_categoria").isNull()).select(df_silver_with_keys['*'])

print(f"    > {df_matched_1.count()} filas categorizadas por coincidencia exacta.")
print(f"    > {df_unmatched_1.count()} filas no se encontraron, pasando al segundo intento.")


# Intento 2: Unir por coincidencia del código sin prefijo (solo para los que fallaron)
print("  - Intento 2: Uniendo por coincidencia del código sin prefijo...")
df_matched_2 = df_unmatched_1.join(
    df_dimensions,
    df_unmatched_1.codigo_sin_prefijo == df_dimensions.codigo_producto,
    "left"
).drop(df_dimensions.codigo_producto) # Se elimina la clave duplicada

# Separamos los que se categorizaron en este segundo intento de los que fallaron definitivamente
df_matched_final_2 = df_matched_2.filter(col("dim_categoria").isNotNull())
df_unmatched_final = df_matched_2.filter(col("dim_categoria").isNull()).select(df_unmatched_1['*'])

print(f"    > {df_matched_final_2.count()} filas categorizadas sin prefijo.")
print(f"    > {df_unmatched_final.count()} filas no se pudieron categorizar.")


# 4. --- Consolidación de Resultados ---
# Unimos los resultados de ambos intentos exitosos
df_categorized = df_matched_1.unionByName(df_matched_final_2)

# Seleccionamos y renombramos las columnas para la tabla final de Oro
df_gold = df_categorized.select(
    "fecha",
    col("comprobante_num").alias("numero_comprobante"),
    "codigo_producto",
    col("dim_nombre_producto").alias("nombre_del_producto"),
    col("dim_categoria").alias("categoria"),
    "cantidad",
    col("precio_un_").alias("precio_unitario"),
    "ganancia",
    "subtotal"
)
print("\nResultados consolidados y listos para guardar.")


# 5. --- Guardado de las Tablas Finales ---
# a) Guardar la tabla de Oro
print(f"\nGuardando la tabla final enriquecida en la Capa de Oro: '{gold_table}'...")
(df_gold.write
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(gold_table))
print("¡Tabla de la Capa de Oro guardada exitosamente!")

# b) Guardar la tabla de reporte con los productos no encontrados
if df_unmatched_final.count() > 0:
    print(f"Guardando {df_unmatched_final.count()} productos no encontrados en la tabla de reporte: '{unmatched_products_table}'...")
    (df_unmatched_final.select("codigo_producto")
     .distinct() # Guardamos solo una vez cada código de producto no encontrado
     .write
     .mode("overwrite")
     .saveAsTable(unmatched_products_table))
    print("¡Tabla de reporte de productos no encontrados guardada exitosamente!")
else:
    print("¡Excelente! Todos los productos fueron categorizados.")


# 6. --- Verificación Final ---
print("\nMostrando una muestra de la tabla final (Capa de Oro):")
display(spark.table(gold_table))

if df_unmatched_final.count() > 0:
    print("\nMostrando una muestra de los productos no encontrados:")
    display(spark.table(unmatched_products_table))


In [0]:
display(spark.table("workspace.tecnomundo_data_processed.reporte_de_ventas_por_articulos_2_cleaned"))

In [0]:
display(spark.table("workspace.tecnomundo_data_raw.reporte_de_ventas_por_articulos_2_raw"))

In [0]:
display(spark.table("workspace.tecnomundo_data_dimensions.category"))